# Data Acquisition

### Import Libraries


In [ ]:
import ee
import geemap
import os
import pandas as pd
import rasterio
import matplotlib.pyplot as plt


current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
work_dir = os.path.join(project_root, 'data', 'raw')
os.makedirs(work_dir, exist_ok=True)

An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


/Users/ozantuncbilek/Library/Python/3.9/lib/python/site-packages/google/api_core/_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.6) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/ozantuncbilek/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
ee.Authenticate()
ee.Initialize(project='manifest-pride-258211')


Successfully saved authorization token.


### Determining Hotest Summer Days

*Using MODIS hottest cloud-free summer days through 10 year*

In [4]:
def get_modis_top_n_hottest_days(start_year=2014, end_year=2024, n_days=3):
    """
    Finds the top N hottest cloud-free summer days for Hamburg using MODIS LST data.
    Returns a DataFrame where each year maps to a list of dictionaries.
    """
    hamburg = ee.Geometry.Point(9.9937, 53.5511).buffer(5000)
    results = {}

    for year in range(start_year, end_year + 1):
        try:
            modis_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
                .filterBounds(hamburg) \
                .filterDate(f'{year}-05-15', f'{year}-09-15')

            def compute_lst(img):
                mean_lst = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=hamburg, scale=1000).get('LST_Day_1km')
                return ee.Feature(None, {'lst': mean_lst, 'date': img.date().format('YYYY-MM-dd')})

            lst_features = modis_collection.map(compute_lst).filter(ee.Filter.notNull(['lst']))

            if lst_features.size().getInfo() == 0:
                continue

            # Instead of .first(), use .limit(n) to get the top N days
            hottest_list = lst_features.sort('lst', False).limit(n_days).getInfo()['features']

            top_days = []
            for feature in hottest_list:
                props = feature['properties']
                if props['lst'] is not None:
                    celsius = props['lst'] * 0.02 - 273.15
                    top_days.append({
                        'date': props['date'],
                        'lst_celsius': round(celsius, 2)
                    })

            results[year] = {'top_days': top_days}
            print(f"Found top {len(top_days)} days for {year}.")

        except Exception as e:
            print(f"Error processing {year}: {str(e)}")
            continue

    return pd.DataFrame.from_dict(results, orient='index')

print("Extracting MODIS data...")
df_top_days = get_modis_top_n_hottest_days()

Extracting MODIS data...
Found top 3 days for 2014.
Found top 3 days for 2015.
Found top 3 days for 2016.
Found top 3 days for 2017.
Found top 3 days for 2018.
Found top 3 days for 2019.
Found top 3 days for 2020.
Found top 3 days for 2021.
Found top 3 days for 2022.
Found top 3 days for 2023.
Found top 3 days for 2024.


In [6]:
for year, row in df_top_days.iterrows():
    print(f"--- Year: {year} ---")

    # Get the list of candidate days for the current year
    top_days_list = row['top_days']

    # Check if the list is not empty
    if not top_days_list:
        print("  No candidate days found.")
        continue

    # Print each candidate day
    for i, day_info in enumerate(top_days_list):
        date = day_info['date']
        temp = day_info['lst_celsius']
        print(f"  {i+1}. Hottest: Date: {date}, Temp: {temp}°C")
    print("-" * 20)

--- Year: 2014 ---
  1. Hottest: Date: 2014-07-04, Temp: 34.83°C
  2. Hottest: Date: 2014-07-19, Temp: 34.75°C
  3. Hottest: Date: 2014-07-20, Temp: 34.27°C
--------------------
--- Year: 2015 ---
  1. Hottest: Date: 2015-07-05, Temp: 38.17°C
  2. Hottest: Date: 2015-07-02, Temp: 34.78°C
  3. Hottest: Date: 2015-06-12, Temp: 34.33°C
--------------------
--- Year: 2016 ---
  1. Hottest: Date: 2016-06-05, Temp: 35.11°C
  2. Hottest: Date: 2016-06-23, Temp: 34.02°C
  3. Hottest: Date: 2016-07-20, Temp: 32.88°C
--------------------
--- Year: 2017 ---
  1. Hottest: Date: 2017-05-27, Temp: 32.62°C
  2. Hottest: Date: 2017-06-02, Temp: 31.26°C
  3. Hottest: Date: 2017-07-09, Temp: 30.79°C
--------------------
--- Year: 2018 ---
  1. Hottest: Date: 2018-07-27, Temp: 37.34°C
  2. Hottest: Date: 2018-07-26, Temp: 37.22°C
  3. Hottest: Date: 2018-08-03, Temp: 37.13°C
--------------------
--- Year: 2019 ---
  1. Hottest: Date: 2019-06-30, Temp: 39.26°C
  2. Hottest: Date: 2019-07-23, Temp: 34.56°C

### Extracting Landsat-8 Images Based on Hottest Days

*Landsat 8 images based on hottest days and calculating LTS*

In [7]:
# Hamburg coordinates, define bounding box for missing tiles issue
hamburg = ee.Geometry.Rectangle([
    9.7,  # Min Longitude
    53.4, # Min Latitude
    10.3, # Max Longitude
    53.7  # Max Latitude
])

def get_robust_landsat_data(target_date_str, max_cloud=20, search_radius_days=15):
    """
    Finds a cloud-free Landsat 8 image for a specific target date.
    Returns the image, the found date, and a status ('SUCCESS', 'FALLBACK', 'FAILURE').
    """
    target_date = ee.Date(target_date_str) # Takes a string date instead of a year
    image_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(hamburg)

    # --- Strategy A: Iterative search for a low-cloud image ---
    for day_offset in range(search_radius_days + 1):
        start_date = target_date.advance(-day_offset, 'day')
        end_date = target_date.advance(day_offset, 'day').advance(1, 'day')

        landsat_collection = image_collection \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUD_COVER', max_cloud)) \
            .sort('CLOUD_COVER')

        if landsat_collection.size().getInfo() > 0:
            found_date_ee = ee.Date(landsat_collection.first().get('system:time_start'))
            found_date_str = found_date_ee.format('YYYY-MM-dd').getInfo()
            image = landsat_collection.mosaic().clip(hamburg)

            def mask_clouds(img):
                qa = img.select('QA_PIXEL')
                cloud_bit_mask = 1 << 3; cloud_shadow_bit_mask = 1 << 4
                mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))
                return img.updateMask(mask)

            # Return with 'SUCCESS' status
            return mask_clouds(image), found_date_str, 'SUCCESS'

    # --- Strategy B: Fallback if no low-cloud image was found ---
    start_date = target_date.advance(-search_radius_days, 'day')
    end_date = target_date.advance(search_radius_days, 'day').advance(1, 'day')

    fallback_collection = image_collection.filterDate(start_date, end_date).sort('CLOUD_COVER')

    if fallback_collection.size().getInfo() > 0:
        best_image = fallback_collection.first()
        cloud_cover_val = best_image.get('CLOUD_COVER').getInfo()

        # Fallback Threshold: Reject images that are excessively cloudy
        if cloud_cover_val > 60: # You can adjust this threshold
             return None, None, 'FAILURE'

        found_date_ee = ee.Date(best_image.get('system:time_start'))
        found_date_str = found_date_ee.format('YYYY-MM-dd').getInfo()
        image = fallback_collection.mosaic().clip(hamburg)

        def mask_clouds(img):
            qa = img.select('QA_PIXEL')
            cloud_bit_mask = 1 << 3; cloud_shadow_bit_mask = 1 << 4
            mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))
            return img.updateMask(mask)

        # Return with 'FALLBACK' status
        return mask_clouds(image), found_date_str, 'FALLBACK'

    # If even the fallback fails, return 'FAILURE'
    return None, None, 'FAILURE'


# Add this helper function to your code
def get_coverage_percentage(image, geometry):
    """Calculates the percentage of valid (unmasked) pixels within a geometry."""
    # Create an image where valid pixels are 1, masked pixels are 0
    valid_pixels = image.select(0).unmask(0).gt(0)

    # Calculate the mean of this binary image within the geometry
    # The mean of a 0/1 image is the percentage of 1s.
    coverage_stats = valid_pixels.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geometry,
        scale=30, # Landsat scale
        maxPixels=1e9
    )

    # The result is a fraction (0 to 1), so multiply by 100
    return ee.Number(coverage_stats.values().get(0)).multiply(100)

# LST calculation function
def calculate_lst(image):
    # Check if input is a valid ee.Image
    if image is None:
        return None

    lst = image.expression(
        '(TIRS1 * 0.00341802 + 149.0) - 273.15',  # Convert Kelvin to Celsius
        {'TIRS1': image.select('ST_B10')}
    ).rename('LST')
    return image.addBands(lst)

In [8]:
years = df_top_days.index.tolist()
lst_images = {}
final_dates = {}

# Process each year one by one
for year in years:
    print(f"\n--- Processing Year: {year} ---")

    # Get the list of hottest candidate days for that year
    top_days_for_year = df_top_days.loc[year, 'top_days']

    successful_candidates = [] # A list to store all successful (non-fallback) results

    # Loop through all candidates for the year to evaluate them
    for i, day_info in enumerate(top_days_for_year):
        target_date = day_info['date']
        print(f"-> Evaluating Candidate #{i+1} | Target Date: {target_date}")

        # Call our robust function to get an image
        landsat_image, found_date, status = get_robust_landsat_data(target_date)

        # We are only interested in high-quality 'SUCCESS' images
        if status == 'SUCCESS':
            # Calculate the percentage of valid pixels for the successful image
            coverage = get_coverage_percentage(landsat_image, hamburg).getInfo()
            print(f"  > Found a SUCCESS candidate. Valid Pixel Coverage: {coverage:.2f}%")

            # Store the result along with its coverage score
            successful_candidates.append({
                'image': landsat_image,
                'found_date': found_date,
                'target_date': target_date,
                'coverage': coverage
            })
        else:
            print(f"  > Candidate resulted in '{status}'. Skipping.")

    # After checking all candidates for the year, decide which one is the best
    if successful_candidates:
        # Sort the successful candidates by their coverage score in descending order
        best_candidate = sorted(successful_candidates, key=lambda x: x['coverage'], reverse=True)[0]

        print(f"BEST OPTION for {year}: Target Date {best_candidate['target_date']} -> Found Image on {best_candidate['found_date']} with {best_candidate['coverage']:.2f}% coverage.")

        # Calculate LST for the best image
        lst_images[year] = calculate_lst(best_candidate['image'])
        final_dates[year] = {
            'target': best_candidate['target_date'],
            'found': best_candidate['found_date'],
            'coverage': best_candidate['coverage']
        }
    else:
        # If no candidates resulted in a 'SUCCESS'
        print(f"FAILURE: Could not find any high-quality image for year {year} from the top candidate days.")


--- Processing Year: 2014 ---
-> Evaluating Candidate #1 | Target Date: 2014-07-04
  > Found a SUCCESS candidate. Valid Pixel Coverage: 99.82%
-> Evaluating Candidate #2 | Target Date: 2014-07-19
  > Found a SUCCESS candidate. Valid Pixel Coverage: 99.82%
-> Evaluating Candidate #3 | Target Date: 2014-07-20
  > Found a SUCCESS candidate. Valid Pixel Coverage: 99.82%
BEST OPTION for 2014: Target Date 2014-07-04 -> Found Image on 2014-07-10 with 99.82% coverage.

--- Processing Year: 2015 ---
-> Evaluating Candidate #1 | Target Date: 2015-07-05
  > Candidate resulted in 'FALLBACK'. Skipping.
-> Evaluating Candidate #2 | Target Date: 2015-07-02
  > Candidate resulted in 'FALLBACK'. Skipping.
-> Evaluating Candidate #3 | Target Date: 2015-06-12
  > Found a SUCCESS candidate. Valid Pixel Coverage: 99.73%
BEST OPTION for 2015: Target Date 2015-06-12 -> Found Image on 2015-06-11 with 99.73% coverage.

--- Processing Year: 2016 ---
-> Evaluating Candidate #1 | Target Date: 2016-06-05
  > Foun

#### Visualize 10 years LTS data for control

In [9]:
Map = geemap.Map(center=[53.55, 9.99], zoom=12)

# Visualization parameters (for single band)
vis_params = {
    'min': 20,  # Min LST (°C)
    'max': 40,  # Max LST (°C)
    'palette': ['blue', 'green', 'yellow', 'red']
}


# Add each year's LST image to the map
for year, lst_image in lst_images.items():
    try:
        lst_single_band = lst_image.select('LST')
        Map.addLayer(lst_single_band, vis_params, f'LST {year}')
    except Exception as e:
        print(f"{year} error: {str(e)}")

# Add layer control panel
Map.addLayerControl()

#map_dir = '/content/drive/MyDrive/UHI-Detection-Analysis/outputs/'
#output_path = map_dir + 'LST_map_10years.html'

project_root = os.path.dirname(os.getcwd())
maps_dir = os.path.join(project_root, 'maps')
os.makedirs(maps_dir, exist_ok=True)
output_path = os.path.join(maps_dir, 'LST_map_10years.html')

Map.to_html(output_path)

Problem in 2017 with 67% coverange and 2024 with 93% coverage few missing pixels

### Extracting Sentinel-2 Images for all summers (Landsat-8 Fallback for missing years)


In [10]:
# Hamburg coordinates, define bounding box for missing tiles issue
hamburg = ee.Geometry.Rectangle([9.7, 53.4, 10.3, 53.7])

# --- LANDSAT 8 IÇIN FONKSIYONLAR ---
def mask_landsat8_clouds(image):
    qa = image.select('QA_PIXEL')
    cloud_bit_mask = 1 << 3
    cloud_shadow_bit_mask = 1 << 4
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))
    return image.updateMask(mask)

def create_landsat_summer_composite(year):
    print(f"\n--- Creating LANDSAT-8 composite for Summer {year} ---")
    start_date = f'{year}-06-01'
    end_date = f'{year}-08-31'

    composite = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterBounds(hamburg) \
        .filterDate(start_date, end_date) \
        .map(mask_landsat8_clouds) \
        .median() \
        .clip(hamburg)
    print(f"Landsat-8 composite successfully created for {year}.")
    return composite

# --- SENTINEL 2 IÇIN FONKSIYONLAR ---
def mask_s2_sr(image):
    scl = image.select('SCL'); good_quality = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6)).Or(scl.eq(7)); return image.updateMask(good_quality)
def mask_s2_l1c(image):
    qa = image.select('QA60'); c=1<<10; s=1<<11; m=qa.bitwiseAnd(c).eq(0).And(qa.bitwiseAnd(s).eq(0)); return image.updateMask(m)

def create_sentinel2_summer_composite(year):
    if year < 2016: return None

    collection_id = "COPERNICUS/S2_SR_HARMONIZED" if year >= 2017 else "COPERNICUS/S2"
    mask_function = mask_s2_sr if year >= 2017 else mask_s2_l1c

    print(f"\n--- Creating SENTINEL-2 composite for Summer {year} ---")
    start_date = f'{year}-06-01'
    end_date = f'{year}-08-31'

    composite = ee.ImageCollection(collection_id) \
        .filterBounds(hamburg) \
        .filterDate(start_date, end_date) \
        .map(mask_function) \
        .median() \
        .clip(hamburg)
    print(f"Sentinel-2 composite successfully created for {year}.")
    return composite

In [11]:
composite_images = {}
years_to_process = range(2014, 2025)

for year in years_to_process:

    # --- THIS IS THE KEY CHANGE: Use Landsat for 2017 as well ---
    if year < 2016 or year == 2017: # Use Landsat for 2014, 2015, AND 2017
        composite = create_landsat_summer_composite(year)
    else: # For all other years (2016, 2018+), use Sentinel-2
        composite = create_sentinel2_summer_composite(year)

    if composite:
        # Check if the composite image actually has data before adding it
        try:
            # A simple check by trying to get a value. If it fails, the image is empty.
            composite.select(0).reduceRegion(ee.Reducer.first(), hamburg, 1000).getInfo()
            composite_images[year] = composite
        except Exception as e:
            print(f"  > WARNING: Composite for {year} was created but appears to be empty. Skipping. Error: {e}")


--- Creating LANDSAT-8 composite for Summer 2014 ---
Landsat-8 composite successfully created for 2014.

--- Creating LANDSAT-8 composite for Summer 2015 ---
Landsat-8 composite successfully created for 2015.

--- Creating SENTINEL-2 composite for Summer 2016 ---
Sentinel-2 composite successfully created for 2016.

--- Creating LANDSAT-8 composite for Summer 2017 ---
Landsat-8 composite successfully created for 2017.

--- Creating SENTINEL-2 composite for Summer 2018 ---
Sentinel-2 composite successfully created for 2018.

--- Creating SENTINEL-2 composite for Summer 2019 ---
Sentinel-2 composite successfully created for 2019.

--- Creating SENTINEL-2 composite for Summer 2020 ---
Sentinel-2 composite successfully created for 2020.

--- Creating SENTINEL-2 composite for Summer 2021 ---
Sentinel-2 composite successfully created for 2021.

--- Creating SENTINEL-2 composite for Summer 2022 ---
Sentinel-2 composite successfully created for 2022.

--- Creating SENTINEL-2 composite for Summ

#### Visualize Sentinel-2 Data

In [12]:
# Helper function to check if an image has valid data
def is_image_valid(image, geometry):
    """Checks if an image contains any unmasked pixels within the geometry."""
    try:
        stats = image.select(0).reduceRegion(reducer=ee.Reducer.mean(), geometry=geometry, scale=100).getInfo()
        return stats and list(stats.values())[0] is not None
    except Exception:
        return False

# Parameters for Sentinel-2
vis_params_s2 = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 100,
    'max': 3500,
    'gamma': 1.4
}

# Corrected parameters for Landsat-8
vis_params_l8_final = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 7000,
    'max': 28000,
    'gamma': 1.4
}

# Create the map
Map = geemap.Map(center=[hamburg.centroid().getInfo()['coordinates'][1], hamburg.centroid().getInfo()['coordinates'][0]], zoom=10)
Map.add_basemap('OpenStreetMap')

for year, image in composite_images.items():
    if not is_image_valid(image, hamburg):
        continue

    # Select the correct parameters and layer name based on the year
    if year < 2016 or year == 2017:
        params = vis_params_l8_final
        layer_name = f'Composite {year} (Landsat-8)'
    else:
        params = vis_params_s2
        layer_name = f'Composite {year} (Sentinel-2)'

    Map.addLayer(image, params, layer_name)

Map.addLayerControl()

# Save the map as an HTML file
#map_dir = '/content/drive/MyDrive/UHI-Detection-Analysis/outputs/'
#output_path = map_dir + 'CompositeSentinel2Landsat8_map_10years.html'

project_root = os.path.dirname(os.getcwd())
maps_dir = os.path.join(project_root, 'maps')
os.makedirs(maps_dir, exist_ok=True)
output_path = os.path.join(maps_dir, 'CompositeSentinel2Landsat8_map_10years.html')

Map.to_html(output_path)

### Export Images as GeoTIF

#### Export Landsat 8 (LTS)

In [ ]:
# 1. Setup Output Directory
project_root = os.path.dirname(os.getcwd())
output_dir = os.path.join(project_root, 'data', 'raw', 'landsat_8')

# Create directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

print(f"Target Directory: {output_dir}")

# 2. Download Loop
for year, lst_image in lst_images.items():
    
    # Define filename
    filename = f"LST_{year}_Hamburg.tif"
    full_path = os.path.join(output_dir, filename)
    
    # Check if file already exists to avoid re-downloading
    if os.path.exists(full_path):
        print(f"Skipping {year}, file already exists.")
        continue

    print(f"Downloading {year} (LST)...")

    try:
        # Export the image locally
        geemap.ee_export_image(
            lst_image.select('LST'),     # Select only the LST band
            filename=full_path,
            scale=30,                    # 30m resolution for Landsat LST
            region=hamburg,              # Region of interest
            file_per_band=False
        )
        print(f"Successfully saved: {filename}")
        
    except Exception as e:
        print(f"Error downloading {year}: {e}")

print("All export tasks completed!")

Target Directory: /Users/ozantuncbilek/Documents/Projects/UHI Detection and Analysis/urban-heat-island/data/raw/landsat_8
Generating URL ...
Please wait ...
Data downloaded to /Users/ozantuncbilek/Documents/Projects/UHI Detection and Analysis/urban-heat-island/data/raw/landsat_8/LST_2014_Hamburg.tif
Successfully saved: LST_2014_Hamburg.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/ozantuncbilek/Documents/Projects/UHI Detection and Analysis/urban-heat-island/data/raw/landsat_8/LST_2015_Hamburg.tif
Successfully saved: LST_2015_Hamburg.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/ozantuncbilek/Documents/Projects/UHI Detection and Analysis/urban-heat-island/data/raw/landsat_8/LST_2016_Hamburg.tif
Successfully saved: LST_2016_Hamburg.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/ozantuncbilek/Documents/Projects/UHI Detection and Analysis/urban-heat-island/data/raw/landsat_8/LST_2017_Hamburg.tif
Successfully saved: LST_2017_Hamburg.ti

#### Export Composite Images (Sentinel 2(10m) - Landsat 8(30m))

In [24]:
import os
import geemap
import ee

# Configuration
OVERWRITE = True 

# 1. Setup Output Directory
project_root = os.path.dirname(os.getcwd())
output_dir = os.path.join(project_root, 'data', 'raw', 'sentinel_2')
os.makedirs(output_dir, exist_ok=True)

print(f"Target Directory: {output_dir}")

# 2. Download Loop
for year, composite_image in composite_images.items():
    
    filename = f"Hamburg_Composite_{year}.tif"
    full_path = os.path.join(output_dir, filename)
    
    if os.path.exists(full_path) and not OVERWRITE:
        print(f"Skipping {year}, file already exists.")
        continue

    try:
        # --- BAND SELECTION ---
        if year < 2016 or year == 2017:
            # LANDSAT 8 (30m)
            selected_image = composite_image.select(
                ['SR_B4', 'SR_B3', 'SR_B2', 'SR_B5', 'SR_B6'],
                ['red', 'green', 'blue', 'nir', 'swir']
            )
            current_scale = 30
            sat_name = "Landsat-8"
        else:
            # SENTINEL 2 (10m)
            selected_image = composite_image.select(
                ['B4', 'B3', 'B2', 'B8', 'B11'],
                ['red', 'green', 'blue', 'nir', 'swir']
            )
            current_scale = 10
            sat_name = "Sentinel-2"

        print(f"Downloading {year} ({sat_name})... This will be split into tiles automatically.")
        
    
        geemap.download_ee_image(
            image=selected_image,
            filename=full_path,
            region=hamburg,
            scale=current_scale,
            crs='EPSG:4326'  
        )
        print(f"Successfully saved {year}")

    except Exception as e:
        print(f"Error downloading {year}: {e}")

print("All downloads completed.")

Target Directory: /Users/ozantuncbilek/Documents/Projects/UHI Detection and Analysis/urban-heat-island/data/raw/sentinel_2


Hamburg_Composite_2014.tif: |          | 0.00/99.4M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


Successfully saved 2014


Hamburg_Composite_2015.tif: |          | 0.00/99.4M (raw) [  0.0%] in 00:00 (eta:     ?)

...

Successfully saved 2022


Hamburg_Composite_2023.tif: |          | 0.00/894M (raw) [  0.0%] in 00:00 (eta:     ?)

Successfully saved 2023


Hamburg_Composite_2024.tif: |          | 0.00/894M (raw) [  0.0%] in 00:00 (eta:     ?)

Successfully saved 2024
All downloads completed.


### Summary and Next Steps

In this notebook, **data acquisition** was prepared to determine the **hottest day** and to download satellite imagery from **Landsat 8 for LST** and **Sentinel-2**.  
These datasets were saved in the `data/raw/` directory.

In the next notebook, **`03_data_processing.ipynb`**, we will combine *LST* with **spectral indices** (e.g., NDVI) to create a **multi-channel tensor**, which will serve as the final input for the **U-Net model**.
